In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
def record_data_info(class_nm, txt_cnt):
    with open('../data/info.txt', 'a') as f:
        f.write(class_nm + ': ' + str(txt_cnt) + '\n')

In [3]:
def get_df():
    data_path = '../data/code25/'
    class_dir_list = [os.listdir(data_path)]

    # count = 0
    
    data_list = []
    class_list = []

    for class_nm in os.listdir(data_path):
        
        class_txt_cnt = 0

        for txt_file in os.listdir(data_path + class_nm):
            if txt_file.endswith('.txt'):
                class_txt_cnt += 1

                file_path = data_path + class_nm + '/' + txt_file

                with open(file_path, 'r') as file:
                    class_label = class_nm

                    strings = file.readlines()
                    data = ' '.join(strings)
                    
                    data_list.append(data)
                    class_list.append(class_label)

                # count += 1
            
            # if count == 10: break
        
        # record_data_info(class_nm, class_txt_cnt)
        # break
    
    data = {'sourceCode': data_list, 'classLabel': class_list}
    df = pd.DataFrame(data)

    return df, data_list, class_list

In [4]:
df, data_list, class_list = get_df()

In [5]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def split(str):
    return [char for char in str]

def tokenize(sourceCode):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_codes = []
    ch2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    ch2idx['<pad>'] = 0

    # Building our vocab from the corpus starting from index 2
    idx = 1
    for code in sourceCode:
        tokenized_code = split(code)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_codes.append(tokenized_code)

        # Add new token to `word2idx`
        for token in tokenized_code:
            if token not in ch2idx:
                ch2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_code))

    return tokenized_codes, ch2idx, max_len

def encode(tokenized_codes, ch2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_code in tokenized_codes:
        # Pad sentences to max_len
        tokenized_code += ['<pad>'] * (max_len - len(tokenized_code))

        # Encode tokens to input_ids
        input_id = [ch2idx.get(token) for token in tokenized_code]
        input_ids.append(input_id)
    
    return np.array(input_ids)

In [6]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_sourceCodes, ch2idx, max_len = tokenize(data_list)
input_ids = encode(tokenized_sourceCodes, ch2idx, max_len)

Tokenizing...

